## Intrinsic value calculator (DCF model)

In [1]:
# Calculate the intrinsic value
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date

In [2]:
ticker = 'BABA'

#### Get data

In [3]:
# Get the latest stored data - annual reports
annuals = glob.glob(f'./annual_reports/{ticker}_*_annual_*.csv')
assert len(annuals) > 0, f'No annual files found for {ticker}'
annuals.sort()
df_annuals = pd.read_csv(annuals[-1], sep=';', header=0, index_col='fiscalYear')

In [4]:
# Get the overview data
overviews = glob.glob(f'./overviews/{ticker}_*_overview_*.csv')
assert len(overviews) > 0, f'No overview files found for {ticker}'
overviews.sort()
df_overviews = pd.read_csv(overviews[-1], sep=';', header=0, index_col='Metric')

#### Functions

In [5]:
# Functions

def calculate_tax_rate(income_tax_expense, income_bef_tax):
    return (income_tax_expense / income_bef_tax)

def calculate_total_debt_ratio(debt, market_cap):
    return debt / (debt + market_cap)

def calculate_market_cap_ratio(debt, market_cap):
    return market_cap / (debt + market_cap)

def calculate_cost_of_debt(interest_expense, total_debt):
    return interest_expense / total_debt

def calculate_cost_of_debt_after_tax(cost_of_debt, tax_rate):
    return cost_of_debt * (1 - tax_rate)

def calculate_cost_of_equity(risk_free_rate, equity_risk_premium, beta):
    return risk_free_rate + (equity_risk_premium * beta)

def calculate_WACC(cost_of_equity, cost_of_debt_after_tax, market_cap_ratio, total_debt_ratio):
    '''Weighted average cost of capital'''
    return (cost_of_equity * market_cap_ratio) + (cost_of_debt_after_tax * total_debt_ratio)

def calc_xnpv(rate, values, dates):
    '''Equivalent of Excel's XNPV function.
       Based on: https://stackoverflow.com/a/33260133'''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])
    
def calc_growth_rate(val_start, val_end):
    return (val_end / val_start) - 1

def get_avg_growth_values(n_periods, growth_rate, val_start):
    out_vals = []
    for period in range(n_periods):
        if period == 0:
            val = val_start
        val = val + (val * growth_rate)
        out_vals.append(val)
    return out_vals

def calc_perpetual_growth_val(ending_FCF_val, perp_growth_rate, discount_rate):
    return (ending_FCF_val * (1 + perp_growth_rate)) / (discount_rate - perp_growth_rate)

def bil(num):
    return round(num / 1000000000, 2)

#### Calculations

In [6]:
# Assumptions

# General

# The reasonable rate (%) that cash flow can grow
perpetual_growth_rate = 0.025
# US gov bond 10y yield
risk_free_rate = 0.013
equity_risk_premium = 0.075

# Company-specific
EV_EBITDA = df_overviews.loc['EVToEBITDA'].astype('float')[0]
shares_outstanding = df_overviews.loc['SharesOutstanding'].astype('int64')[0]
market_capitalization = int(df_overviews.loc['MarketCapitalization', 'Value'])
total_debt = df_annuals.loc[df_annuals.index.max(), ['shortTermDebt', 'longTermDebt']].sum()
cash = df_annuals.loc[df_annuals.index.max(), ['cashAndShortTermInvestments']].astype('float')[0]
capex = df_annuals.loc[:, ['capitalExpenditures']].mean()[0]
beta = df_overviews.loc['Beta',:].astype('float')[0]

# WACC Calculations
tax_rate = calculate_tax_rate(abs(df_annuals.loc[df_annuals.index.max(), 'incomeTaxExpense']),
                              df_annuals.loc[df_annuals.index.max(), 'incomeBeforeTax'])
market_cap_r = calculate_market_cap_ratio(total_debt, market_capitalization)
total_debt_r = calculate_total_debt_ratio(total_debt, market_capitalization)
cost_of_equity = calculate_cost_of_equity(risk_free_rate, equity_risk_premium, beta)
cost_of_debt = calculate_cost_of_debt(interest_expense=abs(df_annuals.loc[df_annuals.index.max(), 'interestExpense']),
                                      total_debt=total_debt)
cost_of_debt_after_tax = calculate_cost_of_debt_after_tax(cost_of_debt, tax_rate)


discount_rate = calculate_WACC(cost_of_equity,
                               cost_of_debt_after_tax,
                               market_cap_r,
                               total_debt_r)

print(f'Market cap: {bil(market_capitalization)}B')
print(f'Total debt: {bil(total_debt)}B')
print(f'Cash: {bil(cash)}B')
print(f'Discount rate: {round(discount_rate*100, 2)}%')

Market cap: 631.9B
Total debt: 125.43B
Cash: 330.5B
Discount rate: 7.21%


#### Future estimations

In [7]:
# EBIT avg growth rate for the past 4 years
ebit_avg_growth = np.mean(df_annuals.loc[:, 'ebit'].sort_index().pct_change().dropna().to_list())
future_ebit_vals = pd.Series(get_avg_growth_values(4, ebit_avg_growth, df_annuals.loc[df_annuals.index.max(), 'ebit']))

future_cash_taxes = future_ebit_vals * tax_rate
DA_avg = np.mean(df_annuals.loc[:, 'depreciationAndAmortization'].abs())
future_NWC = np.mean(pd.Series(df_annuals.loc[:, 'totalCurrentAssets']) - pd.Series(df_annuals.loc[:, 'totalCurrentLiabilities']))

In [8]:
# Next five years
next_5Y = list(range(int(df_annuals.index.max()) + 1, df_annuals.index.max() + 5)) 

df_dcf = future_ebit_vals.to_frame(name='EBIT')
df_dcf.index = next_5Y
df_dcf['Cash taxes'] = df_dcf.loc[:, 'EBIT'] * tax_rate
df_dcf['Depr&Amort'] = np.mean(df_annuals.loc[:, 'depreciationAndAmortization'].abs())
df_dcf['Capex'] = capex
# This needs to be change in NWC, not NWC!
df_dcf['NetWorkCap'] = np.mean((pd.Series(df_annuals.loc[:, 'totalCurrentAssets']) - pd.Series(df_annuals.loc[:, 'totalCurrentLiabilities'])).diff().dropna())
df_dcf['unleveredFCF'] = (df_dcf['EBIT'] - df_dcf['Cash taxes'] + df_dcf['Depr&Amort'] - df_dcf['Capex'] - df_dcf['NetWorkCap'])

In [43]:
# def get_previous_year()

print(df_annuals.loc[:, 'ebit'])
print(df_annuals.loc[:, 'ebit'].rolling(2).sum())
print(df_annuals.loc[2019, 'ebit'] + df_annuals.loc[2020, 'ebit'])
print(df_annuals.loc[:, 'ebit'].diff())
print(df_annuals.loc[2020, 'ebit'] - df_annuals.loc[2019, 'ebit'])   

fiscalYear
2016    1.531890e+03
2017    7.699200e+10
2018    1.259890e+11
2019    1.384910e+11
2020    2.142520e+11
Name: ebit, dtype: float64
fiscalYear
2016             NaN
2017    7.699200e+10
2018    2.029810e+11
2019    2.644800e+11
2020    3.527430e+11
Name: ebit, dtype: float64
352743000000.0
fiscalYear
2016             NaN
2017    7.699200e+10
2018    4.899700e+10
2019    1.250200e+10
2020    7.576100e+10
Name: ebit, dtype: float64
75761000000.0


In [9]:
perpetual_growth = calc_perpetual_growth_val(ending_FCF_val=df_dcf.loc[df_dcf.index.max(),'unleveredFCF'],
                                            perp_growth_rate=perpetual_growth_rate,
                                            discount_rate=discount_rate)
EV_EBITDA_val = EV_EBITDA * (df_dcf.loc[df_dcf.index.max(),'EBIT'] + df_dcf.loc[df_dcf.index.max(),'Depr&Amort'])
exit_FCF = np.mean([perpetual_growth, EV_EBITDA_val])

#### Intrinsic value

In [10]:
# Intrinsic value

transaction_CF = df_dcf['unleveredFCF'].tolist()
transaction_CF.append(exit_FCF)

dates_CF = [date(year, 12, 31) for year in  list(df_dcf.index)]
dates_CF.append(dates_CF[-1])

enterprise_val_intrinsic = calc_xnpv(rate=discount_rate,
                                     values=transaction_CF,
                                     dates=dates_CF)

equity_val_intrinsic = enterprise_val_intrinsic + cash - total_debt
equity_val_per_share_intrinsic = equity_val_intrinsic / shares_outstanding

In [11]:
# Market value (end of last fiscal)
equity_val_market = market_capitalization + cash - total_debt
equity_val_per_share_market = equity_val_market / shares_outstanding

In [12]:
# Intrinsic vs. market
MoS = 0.2
MoS_intrinsic = equity_val_per_share_intrinsic - (equity_val_per_share_intrinsic * MoS)

print(f'Market value per share for {ticker}: {round(equity_val_per_share_market, 2)}')
print(f'Intrinsic value per share for {ticker}: {round(equity_val_per_share_intrinsic, 2)}')

if equity_val_per_share_intrinsic > equity_val_per_share_market:
    print(f'The upside is {round((equity_val_per_share_intrinsic/equity_val_per_share_market -1) * 100, 2)}%')
    print(f'Intrinsic including margin of safety ({MoS * 100}%): {round(MoS_intrinsic, 2)}')
    if MoS_intrinsic > equity_val_per_share_market:
        print(f'That\'s still {round((MoS_intrinsic/equity_val_per_share_market -1) * 100, 2)}% upside')
    else:
        print(f'The company is {round((equity_val_per_share_market/MoS_intrinsic -1) * 100, 2)}% above MoS value')
else:
    print(f'The company is {round((equity_val_per_share_market/equity_val_per_share_intrinsic -1) * 100, 2)}% above intrinsic value')

Market value per share for BABA: 309.34
Intrinsic value per share for BABA: 1.9218587535609333e+31
The upside is 6.212719018284908e+30%
Intrinsic including margin of safety (20.0%): 1.5374870028487465e+31
That's still 4.9701752146279265e+30% upside
